# DSP Algorithm Prototyping

The objective of this notebook is to test your C-based VAD and DOA algorithm using Python. Instead of using the C language directly, the use of Python code can simplify:

* the generation of audio signals, 
* the import/export of audio file,
* the debugging of your C code (plotting, logging, ...)

In [1]:
import ctypes
import numpy as np
import matplotlib.pyplot as plt
from numpy.ctypeslib import ndpointer
from scipy.io import wavfile

## Signal Model

The first objective is to synthesize a stereo random signal using Python. The considered signal is described by the following statistical model

\begin{align}
x_1[n] &= b[n]\\
x_2[n] &= b[n+\tau]
\end{align}

where 

* $b[n]\sim \mathcal{N}(0, \sigma^2)$ is a Gaussian noise with zero mean and variance $\sigma^2$. The noise is assumed white i.e. 

$$E[b[n]b[m]]=\sigma^2 \delta[n-m]=\left\{\begin{array}{cc} \sigma^2 &\text{ if }n=m\\
0 & \text{elsewhere}\end{array}\right.$$

* $\tau \in \mathbb{N}$ is the time delay between the left signal $x_1[n]$ and the right signal $x_2[n]$. 

##### Question [signal model]

* Create the stereo random signal model by implementing the `create_signal` function

In [2]:
def create_signal(tau, sigma, N, type = np.int16):
    """Create a stereo random signal using the white Gaussian noise model and return it using the int16 format"""
    X = np.zeros((2, N))
    # implement the signal model
    # HERE
    X = X.astype(type) 
    return X

The following code calls the `create_signal` and saves it in tne `wav` folder.

In [3]:
tau = 3 # samples
sigma = 2000
Fs = 16000 # Hz
N = 1600
filename = "wav/noise.wav"
X = create_signal(tau, sigma, N)
wavfile.write(filename, Fs, np.transpose(X))

## Python / C Interface

When your final objective is to implement a code in a uP, it is usually more direct to implement your algorithm using the C language. In this section, we show how to test your C code using Python. 

### DSP library

All the C codes are implemented in the `dsp.c` library. The `dsp` C library contains the definition of three functions :

* `print_pcm`: display the content of an audio signal,
* `compute_energy`: compute the energy of an audio signal, 
* `compute_xcorr`: compute the cross correlation of an stereo audio signal

These functions takes as inputs the the stereo audio signal encoded as a 1D array. The structure of the 1D array is provided below : 


| x_1[0] | x_2[0] | x_1[1] | x_2[1] | x_1[2] | x_2[2] | ... | x_1[M-1] | x_2[M-1] |
|---|---|---|---|---|---|---|---|---|


where the samples `x_l[n]` are encoded as `uint16_t` data.

The following Python function shows how to load a stereo wav file and format it as a 1D array of `uint16_t` data.

In [4]:
def load_audio2pcm(filename):
    """This function load a stereo audio file and convert the signal to a 1D signal with uint16 data type """
    Fs, X = wavfile.read(filename)
    x = np.ravel(X, order='C')  # convert 2D signal to 1D signal
    x = x.astype(np.uint16)  # convert to uint16 data
    return Fs, x

### Load the C Library

The C library can be compiled using the following command :

```
gcc -fPIC -shared dsp.c -o dsp.so
```

The following command should create the `dsp.so` file. The `dsp.so` file can be imported in a Python script using the `LoadLibrary` function.

In [5]:
# import C function
dsp = ctypes.cdll.LoadLibrary("./dsp.so")

## load functions
print_pcm = dsp.print_pcm
print_pcm.restype = None
print_pcm.argtypes = [ndpointer(ctypes.c_uint16, flags="C_CONTIGUOUS"), ctypes.c_size_t]

compute_energy = dsp.compute_energy
compute_energy.restype = ctypes.c_float
compute_energy.argtypes = [ndpointer(ctypes.c_uint16, flags="C_CONTIGUOUS"), ctypes.c_size_t]

compute_xcorr = dsp.compute_xcorr
compute_xcorr.restype = ctypes.c_float
compute_xcorr.argtypes = [ndpointer(ctypes.c_uint16, flags="C_CONTIGUOUS"), ctypes.c_size_t, ndpointer(ctypes.c_float, flags="C_CONTIGUOUS"), ctypes.c_size_t]

OSError: [WinError 193] %1 n’est pas une application Win32 valide

### Test your C Code

The following script loads a stereo sound file using the `load_audio2pcm` python code and prints the content of the audio signal using the `print_pcm` C function.

In [ ]:
Fs, x = load_audio2pcm(filename)
print_pcm(x, 10)

## VAD Detection

A simple VAD detector is based on the signal energy criterion :

$$\widehat{\epsilon}_l^2 = \frac{1}{N} \sum_{n=0}^{N-1} x_l^2[n]$$

### Theoretical Analysis

For the considered signal model, the theoretical value is equal to : 

$$\epsilon_l^2 = \frac{1}{N}\sum_{n=0}^{N-1} E[x_l^2[n]] = \frac{1}{N}\sum_{n=0}^{N-1} \sigma^2 = \sigma^2 $$


In [ ]:
energy_theo = sigma**2
print("theoretical value: energy={}".format(energy_theo))

### Implementation of the C function

##### Question [compute_energy]

* In the `dsp.c` file, implement the C function `compute_energy`,
* compare the estimated energy with its theoretical value. 

In [ ]:
energy_est = compute_energy(x, len(x))
print("estimated value: energy={}".format(energy_est))

## DOA Estimation

A simple DOA estimator is based on the maximum of the cross correlation. The cross correlation is defined as :

$$\widehat{R}[m] = \frac{1}{N} \sum_{n=0}^{N-1}x_1[n]x_2[n-m]$$

### Theoretical Analysis

For the considered signal model, the cross correlation is theoretically equal to 

\begin{align}
R[m] = \frac{1}{N} \sum_{n=0}^{N-1}E[x_1[n]x_2[n-m]]= \left\{\begin{array}{cc}
\sigma^2 &\text{ if }m=\tau\\
0 &\text{elsewhere}
\end{array}\right .
\end{align}

In [ ]:
tau_max = 10
N_R = 2*tau_max+1
tau_vect = np.arange(-tau_max, tau_max+1)
R_theo = np.zeros(N_R)
R_theo[tau_max+tau] = sigma**2

plt.stem(tau_vect, R_theo)
plt.xlabel("tau [sample]")

### Implementation of the C function

##### Question [compute_xcorr]

* In the `dsp.c` file, implement the C function `compute_xcorr`,
* compare the estimated correlation with its theoretical values.

In [ ]:
R_est = np.zeros(N_R, dtype=np.float32)
compute_xcorr(x, len(x), R_est, N_R)
print(R_est)
